# Data exploration

This notebook includes basic data exploration aimed at gaining familiarity with loading and formatting stock market data

In [ ]:
%load_ext autoreload
%autoreload 2

import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.offline as pyo
import plotly.graph_objects as go
import plotly.express as px

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = "plotly"

### Step 1: Get data for top 10 companies in FTSE100 by market cap

In [5]:
end_date = dt.datetime(2024, 12, 6)
start_date = end_date - dt.timedelta(days=365.25 * 12)

tickers = ["AZN", "SHEL", "HSBA", "ULVR", "REL", "BATS", "RIO", "LSEG", "BP", "GSK"]
tickers = [i + ".L" for i in tickers]  # Need .L for UK equities

# Load data and format it
df = yf.download(tickers, start=start_date, end=end_date).stack()

df.index.names = ["date", "ticker"]
df.columns = pd.Index([col.lower().replace(" ", "_") for col in df.columns], name="price")

df.sort_index(ascending=True, inplace=True)
df

[*********************100%***********************]  10 of 10 completed
/tmp/ipykernel_4704/706543287.py:8: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



price                 adj_close         close          high           low  \
date       ticker                                                           
2012-12-06 AZN.L    1949.052612   2994.500000   3013.500000   2992.000000   
           BATS.L   1604.408813   3252.000000   3290.000000   3251.000000   
           BP.L      213.160217    427.600006    429.312988    424.149994   
           GSK.L     665.652710   1355.399780   1365.977051   1348.852051   
           HSBA.L    333.575592    643.799988    647.200012    639.000000   
...                         ...           ...           ...           ...   
2024-12-05 LSEG.L  11435.000000  11435.000000  11490.000000  11380.000000   
           REL.L    3784.000000   3784.000000   3796.699951   3762.000000   
           RIO.L    4966.000000   4966.000000   5031.000000   4958.000000   
           SHEL.L   2494.000000   2494.000000   2521.500000   2489.500000   
           ULVR.L   4667.000000   4667.000000   4685.000000   4658.000000   

price                      open      volume  
date       ticker                            
2012-12-06 AZN.L    3000.000000   2371228.0  
           BATS.L   3280.000000   2766423.0  
           BP.L      427.000000  19904771.0  
           GSK.L    1362.451294   6593328.0  
           HSBA.L    640.000000  23162220.0  
...                         ...         ...  
2024-12-05 LSEG.L  11455.000000   1584825.0  
           REL.L    3771.000000   2570264.0  
           RIO.L    4986.000000   4073507.0  
           SHEL.L   2514.500000   8422443.0  
           ULVR.L   4683.000000   3840823.0  

[30306 rows x 6 columns]

### Step 2: Isolate the largest company

In [18]:
df_top = df.loc[(slice(None), tickers[0]), :].copy()
df_top.index = df_top.index.droplevel(level=1)
df_top

price,adj_close,close,high,low,open,volume
date,,,,,,
2012-12-06,1949.052612,2994.5,3013.500000,2992.0,3000.0,2371228.0
2012-12-07,1952.957642,3000.5,3003.000000,2975.0,2994.5,2325295.0
2012-12-10,1960.117310,3011.5,3019.000000,2986.5,2995.0,1388173.0
2012-12-11,1971.832886,3029.5,3032.000000,2998.0,3008.0,1284941.0
2012-12-12,1980.294189,3042.5,3065.275879,3029.0,3038.0,3900294.0
...,...,...,...,...,...,...
2024-11-29,10612.000000,10612.0,10612.597656,10532.0,10568.0,2380552.0
2024-12-02,10554.000000,10554.0,10646.000000,10416.0,10596.0,1931093.0
2024-12-03,10790.000000,10790.0,10804.000000,10562.0,10584.0,2193680.0


### Step 3: Compute return and other useful quantities

In [21]:
# Get daily return
df_top["RTN_D1"] = df_top["close"].pct_change(1)

# Get log daily return
df_top["LOGRTN_D1"] = np.log(df_top["close"]).diff(1)

# Repeat above, but shift by 1 to get prediction variable
df_top["RTN_D1_SHIFT-1"] = df_top["close"].pct_change(1).shift(-1)
df_top["LOGRTN_D1_SHIFT-1"] = np.log(df_top["close"]).diff(1).shift(-1)

# Binary prediction variable: Does price go up or down
df_top["RTN_DIR_D1_SHIFT-1"] = (
    df_top["close"].pct_change(1).map(lambda x: 1 if x > 0 else 0).shift(-1)
)

# Drop rows containing NANs
df_top.dropna(inplace=True)

df_top

price,adj_close,close,high,low,open,volume,RTN_D1,LOGRTN_D1,RTN_D1_SHIFT-1,LOGRTN_D1_SHIFT-1,RTN_DIR_D1_SHIFT-1
date,,,,,,,,,,,
2012-12-07,1952.957642,3000.5,3003.000000,2975.0,2994.5,2325295.0,0.002004,0.002002,0.003666,0.003659,1.0
2012-12-10,1960.117310,3011.5,3019.000000,2986.5,2995.0,1388173.0,0.003666,0.003659,0.005977,0.005959,1.0
2012-12-11,1971.832886,3029.5,3032.000000,2998.0,3008.0,1284941.0,0.005977,0.005959,0.004291,0.004282,1.0
2012-12-12,1980.294189,3042.5,3065.275879,3029.0,3038.0,3900294.0,0.004291,0.004282,-0.027609,-0.027997,0.0
2012-12-13,1925.620605,2958.5,3040.500000,2943.0,3039.0,5473957.0,-0.027609,-0.027997,-0.011661,-0.011730,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-28,10594.000000,10594.0,10654.000000,10572.0,10636.0,1606465.0,0.005887,0.005870,0.001699,0.001698,1.0
2024-11-29,10612.000000,10612.0,10612.597656,10532.0,10568.0,2380552.0,0.001699,0.001698,-0.005466,-0.005481,0.0
2024-12-02,10554.000000,10554.0,10646.000000,10416.0,10596.0,1931093.0,-0.005466,-0.005481,0.022361,0.022115,1.0


## Plotting

Make simple candle stick plot

In [ ]:
ticker = tickers[0].replace(".L", "")


fig = go.Figure()

fig.add_trace(
    go.Candlestick(
        x=df_top.index,
        open=df_top["open"],
        high=df_top["high"],
        low=df_top["low"],
        close=df_top["close"],
        name="OHLC",
    ),
)

fig.update_layout(
    title=f"{ticker} historical price chart",
    xaxis_tickfont_size=12,
    yaxis=dict(
        title="Price (£/share)",
        titlefont_size=14,
        tickfont_size=12,
    ),
    autosize=False,
    width=800,
    height=500,
    margin=dict(l=50, r=50, b=50, t=50, pad=2),
    paper_bgcolor="LightSteelBlue",
)

fig.update(layout_xaxis_rangeslider_visible=True)

Plot daily returns

In [ ]:
fig = px.bar(df_top, x=df_top.index, y="RTN_D1")

fig.update_traces(
    marker_color="rgba(0,0,250, 0.5)", marker_line_width=0, selector=dict(type="bar")
)

fig.update_layout(
    bargap=0,
    bargroupgap=0,
)

fig.update_layout(
    title=f"{ticker} daily returns",
    xaxis_tickfont_size=12,
    yaxis=dict(
        title="Daily return",
        titlefont_size=14,
        tickfont_size=12,
    ),
    autosize=False,
    width=800,
    height=500,
    margin=dict(l=50, r=50, b=50, t=50, pad=2),
    paper_bgcolor="LightSteelBlue",
)

fig.show()